# Dataframe Editing

### 필요 라이브러리 임포트

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer()

### 데이터 불러오기 및 첫 정제

In [2]:
filedir = './data/news_df_210222_v05.xlsx'
#-------------------------------------------------------
#-------------------------------------------------------
# loading crawled csv
def loadexcel(filedir):
    crawldf = pd.read_excel (filedir)
    crawldf = crawldf.dropna()
    # there are 5columns in CrawledDF -> 'media', 'date', 'title', 'article_original', 'url'
    firstdf = crawldf[['media','date','title','article_original','url']].copy()
    # reseting index in order after droping NA
    firstdf = firstdf.reset_index(drop=True)
    return firstdf
    
    

In [3]:
firstdf = loadexcel(filedir)
firstdf

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계전 지구를 홀린 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정경자년 새해에도 K팝 스타들의 세계 무...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
848,한겨레,2020-12-26,방탄학BTSology BTS 연구는 이미 시작되었다,방탄소년단BTS의 전세계적인 인기는 이제 낯설지 않다. 2017년 처음 미국 무대에...,https://news.naver.com/main/read.nhn?mode=LSD&...
849,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 오...,https://news.naver.com/main/read.nhn?mode=LSD&...
850,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년 잊을 수 없는 문화계의 10일 2020년은 한국 문화계의 저력과 바닥이 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
851,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 이미 최선을 다 하고 계신데 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


### 토크나이징

#### okt.nouns

In [ ]:
# from konlpy.tag import Okt
# okt =Okt()

# StopWordsDir = "./data/stopwords.txt"

# # loading stopwords from StopWordsDirectory
# def stopwords(StopWordsDir):
#     with open (StopWordsDir, "r", encoding = 'utf-8') as myfile:
#         data=myfile.read()
#     stop_words=data.split('\n')
    
#     return stop_words
    
# # tokenize without stopwords function   
# def cleantokenize(df):
#     result = []
    
#     for w in okt.nouns(df):
#         if w not in stopwords(StopWordsDir):
#             result.append(w)
    
#     return result

# # apply cleantokenize function to a dataframe
# def applytokenize(df):
#     df['tokenized_sents'] = df.apply(lambda row: cleantokenize(row['article_original']), axis=1)
#     return df

# tokenized_df = applytokenize(firstdf)
# tokenized_df

#### Rhino

In [ ]:
import rhinoMorph
filedir = './data/news_df_210205_v04.xlsx'
StopWordsDir = "./data/stopwords.txt"

# loading stopwords from StopWordsDirectory
def stopwords(StopWordsDir):
    with open (StopWordsDir, "r", encoding = 'utf-8') as myfile:
        data=myfile.read()
    stop_words=data.split('\n')
    
    return stop_words

rhino = rhinoMorph.startRhino()
firstdf['tokenized_sents'] = firstdf.article_original.apply(lambda x: rhinoMorph.onlyMorph_list(rhino, x, pos=['NNG','NNP','VA'], combineN=True)) # eomi=True, xrVv=True))
firstdf.tokenized_sents = firstdf.tokenized_sents.apply(lambda x : [w for w in x if x not in stopwords(StopWordsDir)if len(w) >1])  # 불용어 제거

tokenized_df = firstdf

In [5]:
tokenized_df.head()

,media,date,title,article_original,url,tokenized_sents
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계전 지구를 홀린 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방송, 라이브, 생중계, 지구, 그룹, 사회자, 소개, 방탄, 모습, 마지막, 미..."
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방탄, 미국, 선정, 음악, 변화, 아티스트, 자신, 음악, 장르, 스스로, 재창..."
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정경자년 새해에도 K팝 스타들의 세계 무...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[트와이스, 도쿄돔, 공연, 블랙핑크, 앨범, 예정경자, 새해, 스타, 세계, 무대..."
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[관심, 세계적, 서울시, 명소, 한국, 외국인, 관광객, 트와이스, 스타, 흔적,..."
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[골든글로브, 시상식, 한국, 최초, 외국어영화상, 수상, 기생충, 봉준호, 감독,..."


### 문서 유사도

In [13]:
# 코사인 유사도를 통한 유사한 기사 뽑아내는 함수
def getcosine (df):
    article_data =df.astype(str)
    tfidf_matrix = tfidf.fit_transform(article_data)
    # 코사인유사도 공식
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_sim

# 각 인덱스에 해당하는 기사와 유사한 기사들의 인덱스 추출
def similarity(index,df):
    result= []
    cosine_sim=getcosine(df)
 
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 비슷한 유사도에 해당하는 기사 인덱스 9개 추출
    sim_scores = sim_scores[1:10]
    article_indices = [i[0] for i in sim_scores]
    # 날짜가 너무 퍼지지 않게 Index 범위 조정
    for n in article_indices:
        if n < (int(index)+30) and n > (int(index)-30):
            result.append(n)
    return result

# 해당 컬럼에 해당하는 모든 열에 대하여 수행
def similarlist (df):
    new_list = []
    for i in range(len(df)):
        new_list.append(similarity(i,df))
    return new_list

# 유사도 시각화를 위한 컬럼 추가
def applysim (df):
    df['common'] = similarlist(df['article_original'])
    # List 로 된 Similar Article String 타입으로
    df['sim'] = [', '.join(map(str, l)) for l in df['common']]
    return df

In [14]:
applysim_df = applysim (tokenized_df)
applysim_df.head()

,media,date,title,article_original,url,tokenized_sents,common,sim
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계전 지구를 홀린 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방송, 라이브, 생중계, 지구, 그룹, 사회자, 소개, 방탄, 모습, 마지막, 미...","[1, 2]","1, 2"
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방탄, 미국, 선정, 음악, 변화, 아티스트, 자신, 음악, 장르, 스스로, 재창...",[0],0
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정경자년 새해에도 K팝 스타들의 세계 무...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[트와이스, 도쿄돔, 공연, 블랙핑크, 앨범, 예정경자, 새해, 스타, 세계, 무대...","[19, 8, 9, 11, 0]","19, 8, 9, 11, 0"
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[관심, 세계적, 서울시, 명소, 한국, 외국인, 관광객, 트와이스, 스타, 흔적,...",[],
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[골든글로브, 시상식, 한국, 최초, 외국어영화상, 수상, 기생충, 봉준호, 감독,...",[],


### 데이터프레임 최종 정제

In [15]:
# 존재했던 데이터프레임에서 유사도가 겹치지 않는 기사들을 drop, 그리고 재정렬 
# 재정렬하면 유사도 인덱스또한 변동 사항이 때문에 다시 받아야한다
def recleanse(df):
    newdf = df.drop(columns = 'common')
    newdf = newdf.replace('', np.nan)
    newdf = newdf.dropna()
    newdf = newdf.drop(columns = 'sim')
    newdf = newdf.reset_index (drop=True)
    return newdf

In [16]:
cleansed_df = recleanse(applysim_df)

In [17]:
def recosinesim(df):
    df['sim'] = similarlist(df['tokenized_sents'])
    df['sim'] = [', '.join(map(str, l)) for l in df['sim']]
    
    simcount = pd.Series(df['sim'].str.count('\w+'), name = 'simcount')
    finaldf = pd.concat([df, simcount], axis=1)
    return finaldf


In [ ]:
final_df = recosinesim(cleansed_df)

In [23]:
final_df.head()

,media,date,title,article_original,url,tokenized_sents,sim,simcount
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계전 지구를 홀린 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방송, 라이브, 생중계, 지구, 그룹, 사회자, 소개, 방탄, 모습, 마지막, 미...","1, 2",2
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방탄, 미국, 선정, 음악, 변화, 아티스트, 자신, 음악, 장르, 스스로, 재창...",0,1
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정경자년 새해에도 K팝 스타들의 세계 무...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[트와이스, 도쿄돔, 공연, 블랙핑크, 앨범, 예정경자, 새해, 스타, 세계, 무대...","8, 26, 5, 25, 6, 13, 7",7
3,조선일보,2020-01-07,BTS 상표권 분쟁 일단락 신세계 상표권 포기 방탄소년단 응원,BTS 상표권을 두고 그룹 방탄소년단BTS 소속사 빅히트엔터테인먼트이하 빅히트와 분...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[상표, 그룹, 소속사, 빅히트엔터테인먼트이하, 빅히트, 분쟁, 신세계, 상표, 포...",4,1
4,한겨레,2020-01-07,신세계 BTS 상표권 모두 포기,방탄소년단BTS 소속사 빅히트엔터테인먼트와 비티에스BTS 상표권을 두고 분쟁 중이었...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[소속사, 빅히트엔터테인먼트, 상표, 분쟁, 신세계백화점, 관련, 상표, 신세계백화...",3,1


### Clustering, Summarization 단계를 위해 저장

In [13]:
# 내 마음속에 저장
savedir = './data/will_rhino_cosine.csv'
def dftocsv (final_df, savedir):
    final_df.to_csv (savedir, index = False, encoding = 'utf-8-sig')
    # 쓸데없는 (유사도가 겹치지않는) 기사들을 제거 한 최종 데이터 프레임
    
dftocsv (final_df, savedir)